# Projeto Final

## Integrantes
* Cynara Valeria de Oliveira Costa do Amaral Cavalcanti
* Lucas Gabriel Rodrigues de Melo
* Raissa Heimman
* Thays Vithorya Cristovam Cipriano
* Victor Miguel de Morais Costa
* Weslley Batista da Hora

## Instalação de Pacotes

In [1]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.8 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl siz

In [2]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 2.8 MB/s eta 0:00:00


In [3]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft

  Cloning https://github.com/facebookresearch/audiocraft to /tmp/pip-install-3kjj4e6w/audiocraft_f82ba3b0c2ac485fa6bab2a961a72476
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft /tmp/pip-install-3kjj4e6w/audiocraft_f82ba3b0c2ac485fa6bab2a961a72476
  Resolved https://github.com/facebookresearch/audiocraft to commit 69fea8b290ad1b4b40d28f92d1dfc0ab01dbab85
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.6 MB/s eta 0:00:00
  Prepari

## Importação de Pacotes

In [4]:
import torch
import numpy
import scipy
import pathlib
import textwrap
import PIL.Image
import gradio as gr
from google.colab import userdata
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio

In [6]:
import vertexai
from vertexai.preview.vision_models import Image, ImageGenerationModel

from google.colab import auth

gcp_project_id = userdata.get("GCP_PROJECT_ID")

auth.authenticate_user(project_id=gcp_project_id)

vertexai.init(project=gcp_project_id)

## Implementação do App

In [7]:
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

genai.configure(api_key=GEMINI_API_KEY)

In [8]:
class MusicGenWrapper:
    def __init__(self):
        self.text_to_text_model = genai.GenerativeModel("gemini-pro")
        self.modelMusic = musicgen.MusicGen.get_pretrained('small', device='cuda')
        self.modelMusic.set_generation_params(duration=60)

        return

    def text_to_song(self, single_description, image_feeling_sentiment_vibe_description):
        final_prompt=f"""
        I need to generate a prompt for an AI that generates music given the descriptions below. You need to combine these two descriptions in order to generate a prompt must be
        very detailed about how a song should.

        Given the following description of a song: {single_description}
        And given the following description of the vibes/sentiments/feelings that one gets from viewing an image: {image_feeling_sentiment_vibe_description}

        I'm going to give you some example prompts and I need you to generate one for me completely according to the description I gave.

        The output should be ONLY about these topics with as most details as possible and NOTHING else:
        - Song melody. Chord progression. Genre and mood. Which musical instruments are present and how they are being played. Tempo and rhythm. Musical elements.

        Example 1:
        (Request):
        Song Description: An energetic song that has use of complex instruments. It uses lots of instruments and is fast-paced.
        Feelings, Sentiments and Vibes caught by the Image Description: The image conveys a feeling of adventure and exploration of the unknown. It also passes and intrigue and misterious vibe with action also.

        (Response):
        An upbeat and futuristic melody that uses electronic sounds. A complex chord progression that includes both major and minor chords. Electronic dance music with a high-energy and optimistic mood.
        Synthesizers, drums, and bass. Fast and pulsating with a driving rhythm. Catchy hooks, dynamic drops, and a strong beat.


        Example 2:
        (Request):
        Song Description: A calm song with some beachy vibes that passes a peaceful feeling to the listeners.
        Feelings, Sentiments and Vibes caught by the Image Description: The image conveys the idea of finding inner peace through reflection and medidation. It also has some kind of summer vibe and a feeling of relaxing.

        (Response):
        A laid-back and tropical melody that features steel drums. A simple and repetitive chord progression that creates a relaxed atmosphere. Reggae-inspired pop song with a carefree and summery mood.
        Steel drums, guitar, bass, and drums. Medium tempo with a groovy and syncopated rhythm. Smooth vocals, catchy chorus, and a cheerful vibe.


        Example 3:
        (Request):
        Song Description: A sad song that evokes feelings of melhancoly and nostalgia. The mood passed by the song is about reflection, instropection and it contains light harmonies.
        Feelings, Sentiments and Vibes caught by the Image Description: The image passes a sentiment of desolation, sadness and depression. It also shows an eerie vibe with some elements of melancholy

        (Response):
        A melancholic and haunting melody that evokes a sense of nostalgia. A minor key chord progression that uses unusual chords to create an eerie atmosphere.
        Indie folk with a bittersweet and reflective mood. Acoustic guitar, piano, strings, and percussion. Slow tempo with a waltz-like rhythm. Soulful vocals, intricate harmonies, and poetic lyrics.
        """

        single_file_path = "single.wav"
        final_single_description = self.text_to_text_model.generate_content(final_prompt)
        response = self.modelMusic.generate([final_single_description.text], progress=True)
        single_sampling_rate = self.modelMusic.sample_rate
        scipy.io.wavfile.write(single_file_path, rate=single_sampling_rate, data=response[0, 0].cpu().numpy())

        return single_file_path

In [9]:
class GeminiWrapper:
    def __init__(self):
        self.image_to_text_model = genai.GenerativeModel("gemini-pro-vision")
        self.text_to_text_model = genai.GenerativeModel("gemini-pro")
        self.text_to_image_model = ImageGenerationModel.from_pretrained("imagegeneration@005")

        return

    def image_to_text(self, image):
        image = PIL.Image.fromarray(image)
        description_prompt = """
          Please, provide a very detailed description with at least 200 words, in English,
          about the image that is being sent."""
        response = self.image_to_text_model.generate_content([description_prompt, image])

        return response.text

    def text_to_sentiment(self, image_description):
        image_feeling_sentiment_vibe_prompt = """
        In this request, I am providing an image description to you after the character ':'.
        As a response, I want you to provide to me a possible brief description of the vibes, sentiments and feelings (in English!)
        that one person could get by reading such image description. However, I don't want you to put descriptive elements of the image in your response.
        I want you to focus only on the feelings, sentiments and vibes, as I've already mentioned. Also make sure to not use a bulletpoints list or something like that.
        Instead, describe such a thing with a single sentence"""
        complete_prompt = f"{image_feeling_sentiment_vibe_prompt}: {image_description}"
        response = self.text_to_text_model.generate_content(complete_prompt)

        return response.text

    def text_to_text(self, image_feeling_sentiment_vibe_description, single_description):
        single_vibe_prompt = """
        In this request, I am providing you a detailed description of a song. Such description might contain a lot of different elements
        (such as which musical instruments are being used in the song, how they are being played, the intensity and rythm of each instruments, influences from other artists, bands and/or musical genres).
        The description will start after the next ':' character. As a response, I want you to provide to me a possible brief description of the vibes, sentiments and feeling (in English!!) that one person
        could get by reading such description. I want you to focus ONLY on the feelings, sentiments and vibes, as I've already mentioned. Moreover, make sure to not use a bulletpoints list or something like that in your response.
        Instead, give your response with a single sentence"""
        single_vibe_sentiment = self.text_to_text_model.generate_content(f"{single_vibe_prompt}: {single_description}")

        single_name_prompt = """
        In this request, I am providing you two elements as an ordered list after the character ':'.
        The first element is the description of the vibes, feelings and sentiments that one person get by looking at a certain image.
        The second element is the description of the vibes, feelings and sentiments that one person get reading the description of a certain song.
        As a response, based on these two descriptions, I want you to generate the name of a fictional song that captures the ideas conveyed by these two descriptions.
        Here goes the ordered list of the mentioned elements"""
        final_prompt = f"""
        {single_name_prompt}:
        1) {image_feeling_sentiment_vibe_description}
        2) {single_vibe_sentiment}
        """
        response = self.text_to_text_model.generate_content(final_prompt)

        return response.text


    def sentiment_to_image_vertex(self, single_feeling_sentiment_vibe_description):
        sentiment_to_image_prompt = f"""
        In this request, I am providing one element in the form of an ordered list (a brief description of the feelings, sentiments and vibes that one person could get by listening to a certain song).
        As a response, based on this last brief description, I want you to generate an image with a scenario that conveys the main ideas of the previously mentioned brief description.
        The elements inside the image MUST match the sentiments, feelings and vibes provided in such description.
        Moreover, the scenario must be consistent with the description.
        Ok. Here goes the ordered list of three elements:
        1) Brief description of the feelings, sentiments and vibes of the song: {single_feeling_sentiment_vibe_description}
        """

        images = self.text_to_image_model.generate_images(prompt=sentiment_to_image_prompt)
        images[0].save(location="single_cover.png", include_generation_parameters=True)

        return "single_cover.png"

In [10]:
"""
If your function accepts more than one argument, as is the case above, pass a list of input components to inputs,
with each input component corresponding to one of the arguments of the function, in order.
The same holds true if your function returns more than one value: simply pass in a list of components to outputs.
"""

gemini = GeminiWrapper()
music_gen = MusicGenWrapper()

def generate_single_artifacts(artist_band_name, single_vibe_image, single_description):
    image_description = gemini.image_to_text(single_vibe_image) # Obtém a descrição detalhada (com no mínimo 200 palavras) da imagem de entrada que vai servir para obtenção da vibe do Single.
    image_feeling_sentiment_vibe_description = gemini.text_to_sentiment(image_description) # Obtém a descrição (em poucas palavras) da vibe da imagem de entrada.

    single_name = gemini.text_to_text(image_feeling_sentiment_vibe_description, single_description) # Obtém o nome do Single, dados: Descrição da vibe do Single, Descrição musical do Single.
    single_cover = gemini.sentiment_to_image_vertex(image_feeling_sentiment_vibe_description) # Obtém a capa do Single dados: Nome do artista/banda, Descrição da vibe do Single, Descrição musical do Single.

    single_file_path = music_gen.text_to_song(single_description, image_feeling_sentiment_vibe_description)

    return [artist_band_name, single_name, image_description, image_feeling_sentiment_vibe_description, "single_cover.png", "single.wav"]

# The Interface class is designed to create demos for machine learning models which accept one or more inputs, and return one or more outputs.
apollo_demo = gr.Interface(
    description="""
    # Apollo

    ### O que é?
    * Apollo é uma ferramenta simples para geração de Singles (ou melhor dizendo, 'sneak peeks' de 30 segundos de um Single) que segue as instruções fornecidas pelo usuário.
    * Basta fornecer algumas entradas específicas, explicadas abaixo, e, ao final do processo, você obterá o Single como resultado e poderá baixá-lo.

    ### Entradas
    * __Nome do artista/banda responsável pelo Single:__ Aqui, o usuário pode escolher livremente o nome do artista/banda que irá 'compor' o Single em questão.
    * __Imagem que representa a vibe do Single:__ Uma imagem que será usada para extrair o "sentimento geral" que o Single deseja transmitir para o público.
    * __Descrição musical do Single:__ Uma descrição textual individual a respeito do Single a ser gerado. Aqui, o usuário tem total liberdade para descrever como quer que o single seja. O usuário pode descrever o uso de instrumentos musicais, a maneira como os intrumentos devem ser tocados, a intensidade e ritmo de cada um. Além disso, pode citar influências de gêneros musicais, artistas, bandas. O limite é a sua imaginação! E, lembre-se, quanto mais detalhado, melhor!

    ### Saídas
    * __Nome do Artista:__ O nome do artista/banda que produziu o Single gerado, escolhido cuidadosamente pelo deus Apollo pessoalmente.
    * __Nome do Single:__ O nome do Single gerado, escolhido cuidadosamente pelo deus Apollo pessoalmente.
    * __Capa do Single:__ A capa do Single gerado, ilustrada exclusivamente pelo próprio Apollo.
    * __Single:__ O arquivo do Single gerado no formato '.wav' disponível para download.

    """,
    fn=generate_single_artifacts, # You can pass any function that you want to wrap with a UI. Here, we saw a simple function, but it could be anything from a music generator to the prediction function of a pretrained machine learning model.
    inputs=[
        gr.Textbox(lines=1, max_lines=1, label="Nome do artista/banda responsável pelo Single"),
        gr.Image(label="Imagem que representa a 'vibe' do Single"),
        gr.Textbox(lines=3, max_lines=10, label="Descrição musical do Single"),
    ],
    outputs=[
        gr.Textbox(lines=1, max_lines=1, label="Nome do artista/banda"),
        gr.Textbox(lines=1, max_lines=1, label="Nome do Single gerado"),
        gr.Textbox(lines=3, max_lines=10, label="Descrição da imagem de entrada"),
        gr.Textbox(lines=3, max_lines=10, label="Sentimentos/Emoções/Vibes inferidas da descrição da imagem de entrada"),
        gr.Image(label="Capa do Single"),
        gr.Audio(label="Single")
    ],
    allow_flagging="never",
    clear_btn=gr.Button(visible=False),
    submit_btn=gr.Button(value="Gerar"),
)

apollo_demo.launch(debug=True, share=True)

/usr/local/lib/python3.10/dist-packages/audiocraft/models/musicgen.py:80: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-small
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


state_dict.bin:   0%|          | 0.00/841M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a1509353b69cf1fc37.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a1509353b69cf1fc37.gradio.live
